In [2]:
import pandas as pd
import numpy as np
from datetime import datetime


In [16]:

import pandas as pd

# Importing the dataset
df = pd.DataFrame(pd.read_csv('../raw_data/bakerysales.csv'))

df['date_time'] = df['date'] + ' ' + df['time']
df['date_time'] = pd.to_datetime(df['date_time'])

# Extract the articles and the quantities in order to transform them into column s through a pivot method.
# We'll now have 149 column, one per product with the corresponding qty
pivot = df[['article', 'Quantity']]
products = pivot.pivot(columns = 'article', values = 'Quantity')

# Merge the pivot table with the original dataset and fill the Nan with zeros
# Now for each date point we have the quantity of the article sold
data = df.merge(products, left_index = True, right_index = True)
data = data.fillna(value = 0)

# Keep only the top 7 products (representing 68% of the volume sold)
# Set date as index
data_target = data[['date_time', 'TRADITIONAL BAGUETTE', 'CROISSANT', 'COUPE', 'PAIN AU CHOCOLAT', 'BAGUETTE', 'BANETTE', 'CEREAL BAGUETTE']]
data_target = data_target.resample('20min', on = 'date_time').sum()

data_target = data_target.rename(columns = {'TRADITIONAL BAGUETTE' : 'traditional_baguette',
                                             'CROISSANT' : 'croissant',
                                            'COUPE' : 'coupe',
                                            'PAIN AU CHOCOLAT' : 'pain_au_chocolat',
                                            'BAGUETTE' : 'baguette',
                                            'BANETTE' : 'banette',
                                            'CEREAL BAGUETTE' : 'cereal_baguette'})


# Removing the empty rows
data_target = data_target[data_target != 0]
data_target.dropna(axis = 0, how = 'all', inplace = True)
data_target = data_target.fillna(value = 0)


In [9]:
# Function for converting date in format dd/mm/yyy to yyyy-mm-dd datetime type
def convert_to_date(date_string):
    date_list = date_string.split("/")
    _day, _month, _year = date_list[0], date_list[1], date_list[2]
    date_str = _year+"-"+_month+"-"+_day
    return datetime.strptime(date_str, '%Y-%m-%d').date()

# Load weather data
data_weather = pd.read_csv('../raw_data/paris_weather.csv')
df_weather = data_weather

# Converting date in format dd/mm/yyy to yyyy-mm-dd datetime type
df_weather['date'] = df_weather['DATE'].apply(convert_to_date)
df_weather['date'] = pd.to_datetime(df_weather['date'])


# Creating day of week as a cyclical feature
# First, create the day of the week as a numerical feature
df_weather['day_of_week'] = pd.to_datetime(df_weather['date']).dt.weekday
# Since we have a 7 days week period (e.g., days in a week)
period = 7
# Convert 'day_of_week' to radians
df_weather['day_of_week_radians'] = 2 * np.pi * df_weather['day_of_week'] / period
# Create new features using sine and cosine
df_weather['day_of_week_sin'] = np.sin(df_weather['day_of_week_radians'])
df_weather['day_of_week_cos'] = np.cos(df_weather['day_of_week_radians'])
# Dropping ['month_radians']
df_weather.drop(columns=['day_of_week_radians', 'day_of_week', 'DATE'], inplace=True)


# Creates cyclical month feature according to the date
df_weather['month'] = df_weather.date.dt.month
# Assuming we have a 12 month period (e.g., month in a year)
period = 12
# Convert 'month' to radians
df_weather['month_radians'] = 2 * np.pi * df_weather['month'] / period
# Create new features using sine and cosine
df_weather['month_sin'] = np.sin(df_weather['month_radians'])
df_weather['month_cos'] = np.cos(df_weather['month_radians'])
# Dropping ['month_radians']
df_weather.drop(columns=['month_radians', 'month'], inplace=True)


# Generates periodical features for SUNRISE and SUNSET
# Creates cyclical time feature according to the SUNRISE time
df_weather['sunrise_time'] = pd.to_datetime(df_weather['SUNRISE']).dt.hour*60+pd.to_datetime(df_weather['SUNRISE']).dt.minute
# Assuming we have a 1440 minutes in period (e.g., minutes  in a day)
period = 1440
# Convert 'month' to radians
df_weather['sunrise_time_radians'] = 2 * np.pi * df_weather['sunrise_time'] / period
# Create new features using sine and cosine
df_weather['sunrise_time_sin'] = np.sin(df_weather['sunrise_time_radians'])
df_weather['sunrise_time_cos'] = np.cos(df_weather['sunrise_time_radians'])
# Dropping ['sunrise_time_radians']
df_weather.drop(columns=['sunrise_time_radians', 'sunrise_time', 'SUNRISE'], inplace=True)


# Creates cyclical time feature according to the SUNSET time
df_weather['sunset_time'] = pd.to_datetime(df_weather['SUNSET']).dt.hour*60+pd.to_datetime(df_weather['SUNSET']).dt.minute
# Assuming we have a 1440 minutes in period (e.g., minutes  in a day)
period = 1440
# Convert 'month' to radians
df_weather['sunset_time_radians'] = 2 * np.pi * df_weather['sunset_time'] / period
# Create new features using sine and cosine
df_weather['sunset_time_sin'] = np.sin(df_weather['sunset_time_radians'])
df_weather['sunset_time_cos'] = np.cos(df_weather['sunset_time_radians'])
# Dropping ['sunset_time_radians']
df_weather.drop(columns=['sunset_time_radians', 'sunset_time', 'SUNSET'], inplace=True)


# Setting new date column as index
df_weather.set_index(['date'], inplace=True)
df_weather.head()


# Creating the dictionary for the weather comments and converting OPINION to numbers
dict_meteo = {
'météo idéale': 4,
'météo favorable': 3,
'météo correcte': 2,
'météo défavorable': 1,
'météo très défavorable': 0
}
df_weather['opinion'] = df_weather['OPINION'].map(dict_meteo)
df_weather.drop(columns= ['OPINION'], inplace=True)
df_weather.head()


# Creates dictionary with Holidays
holidays = [
    '2021-1-1',
    '2021-4-5',
    '2021-5-1',
    '2021-5-8',
    '2021-5-13',
    '2021-5-24',
    '2021-7-14',
    '2021-8-15',
    '2021-11-1',
    '2021-11-11',
    '2021-12-25',
    '2022-1-1',
    '2022-4-18',
    '2022-5-1',
    '2022-5-8',
    '2022-5-26',
    '2022-6-6',
    '2022-7-14',
    '2022-8-15',
]
holidays = [pd.to_datetime(holiday)for holiday in holidays ]
# Add isHoliday column to the df_weather
df_weather['isHoliday'] = df_weather.index.map(lambda x: 1 if x in holidays else 0)


/tmp/ipykernel_21591/2798625152.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather['sunrise_time'] = pd.to_datetime(df_weather['SUNRISE']).dt.hour*60+pd.to_datetime(df_weather['SUNRISE']).dt.minute
/tmp/ipykernel_21591/2798625152.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather['sunrise_time'] = pd.to_datetime(df_weather['SUNRISE']).dt.hour*60+pd.to_datetime(df_weather['SUNRISE']).dt.minute
/tmp/ipykernel_21591/2798625152.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather['sunset_time'] = pd.to_datetime(df_weather['SUNSET']).dt.hour*

In [18]:
df_weather.head()


,MAX_TEMPERATURE_C,MIN_TEMPERATURE_C,WINDSPEED_MAX_KMH,TEMPERATURE_MORNING_C,TEMPERATURE_NOON_C,TEMPERATURE_EVENING_C,PRECIP_TOTAL_DAY_MM,HUMIDITY_MAX_PERCENT,VISIBILITY_AVG_KM,PRESSURE_MAX_MB,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,sunrise_time_sin,sunrise_time_cos,sunset_time_sin,sunset_time_cos,opinion,isHoliday
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01,3,2,7,2,3,3,0.5,86,7.750,1014,...,-0.433884,-0.900969,0.5,0.866025,0.754710,-0.656059,-0.971342,-0.237686,0,1
2021-01-02,5,-1,7,2,4,4,0.1,76,10.000,1015,...,-0.974928,-0.222521,0.5,0.866025,0.754710,-0.656059,-0.972370,-0.233445,0,0
2021-01-03,4,2,10,3,3,3,0.3,92,10.000,1016,...,-0.781831,0.623490,0.5,0.866025,0.754710,-0.656059,-0.973379,-0.229200,0,0
2021-01-04,3,2,13,2,2,3,1.0,95,6.875,1015,...,0.000000,1.000000,0.5,0.866025,0.754710,-0.656059,-0.974370,-0.224951,0,0
2021-01-05,3,2,14,2,2,3,0.4,94,7.750,1017,...,0.781831,0.623490,0.5,0.866025,0.757565,-0.652760,-0.975342,-0.220697,0,0


In [22]:
merged_data = data_target.join(df_weather, how = 'outer')

final_data = merged_data.drop(['WINDSPEED_MAX_KMH',
                           'PRECIP_TOTAL_DAY_MM',
                           'HUMIDITY_MAX_PERCENT',
                           'VISIBILITY_AVG_KM',
                           'PRESSURE_MAX_MB',
                           'CLOUDCOVER_AVG_PERCENT',
                           'DEWPOINT_MAX_C',
                           'WEATHER_CODE_MORNING',
                           'WEATHER_CODE_NOON',
                           'WEATHER_CODE_EVENING',
                           'TOTAL_SNOW_MM',
                           'SUNHOUR',
                           'day_of_week_sin',
                           'day_of_week_cos',
                           'isHoliday'], axis = 1)

final_data = final_data.fillna(value = 0)


In [24]:
final_data.to_csv('../raw_data/final_dataset.csv', index=False)
